In [15]:
import sys
import os

#add pde_expr module to python path environment dynamically
dir_path = os.getcwd()
dir_path = os.path.dirname(dir_path)
print(dir_path)
sys.path.append(dir_path)

/home/aben-ham/Desktop/work/stage/my_manapy


In [16]:
# All user front end interface => method and functions are located at Symbol module

In [17]:
from pde_expr.Symbol import Symbol, dx, dy, dz, dt, dx2, dy2, dz2, laplace, grad, divergence

# creating the variables and constants
a, b, c, d = Symbol.symbols("a b c d", sb_type = Symbol.Variable)
K, G, L, Q , Alpha, Beta = Symbol.symbols("K G L Q alpha beta", sb_type = Symbol.Constant)

In [19]:
expr = grad(grad(a + b) * a)

expr = expr.decompose()
print(expr.as_string())
expr.print()


Grad(a).Grad(a) + a.Grad(Grad(a)) + Grad(b).Grad(a) + a.Grad(Grad(b))
                                                                                                                            OP[Add]                                                                                                                             
                                                                ╭──────────────────────────────────────────────────────────────┴──────────────────────────────────────────────────────────────╮                                                                 
                                                            OP[Add]                                                                                                                         OP[Mul]                                                             
                                ╭──────────────────────────────┴──────────────────────────────╮                                                                

'(Grad(a))(Grad(a)) + (a)(Grad(Grad(a))) + (Grad(b))(Grad(a)) + (a)(Grad(Grad(b)))'

In [5]:
# test different expression and see if the result is as expected
#expr = grad(c * (a + b))
expr = grad(a * grad(b * c))
#expr = dt(a) + K * grad(a)
print(expr.as_string())

decompose_expr = expr.decompose()
print(decompose_expr.as_string())

# to print the tree
decompose_expr.print()

Grad(a.Grad(b.c))
a.b.Grad(Grad(c)) + a.Grad(c).Grad(b) + b.Grad(c).Grad(a) + a.c.Grad(Grad(b)) + a.Grad(b).Grad(c) + c.Grad(b).Grad(a)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            OP[Add]                                                                                                                                                                                                                                                                                                                                                             

In [6]:
# check for equality
def is_equal(a, b):
  res = a.decompose() == b.decompose()
  e = "\033[32mequal to\033[0m" if res else "\033[31mnot equal to\033[0m"
  print(f"{a.as_string()} {e} {b.as_string()}")


In [7]:

is_equal(grad(a + b), grad(a) + grad(b))
is_equal(grad(a * b), a * grad(b) + b * grad(a))
is_equal(grad(K * a + L * b), K * grad(a) + L * grad(b))

print()

is_equal(divergence(a + b) , divergence(a) + divergence(b))
is_equal(divergence(Alpha * a) , Alpha * divergence(a))
is_equal(divergence(Alpha * a + Beta * b) , Alpha * divergence(a) + Beta * divergence(b))



Grad(a + b) equal to Grad(a) + Grad(b)
Grad(a.b) equal to a.Grad(b) + b.Grad(a)
Grad(K.a + L.b) equal to K.Grad(a) + L.Grad(b)

Divergence(a + b) equal to Divergence(a) + Divergence(b)
Divergence(α.a) equal to α.Divergence(a)
Divergence(α.a + β.b) equal to α.Divergence(a) + β.Divergence(b)


In [8]:
is_equal(a , b)
is_equal(a * b , 2 * a * b)
is_equal(a + b , b + a)
is_equal(2 * (a + b) , 2 * b + 2 * a)
is_equal(a * (a + b) , a * a + a * b)

#The possibility of simplifying the expression and then comparing them does not exist
is_equal((a + b) * (a + b) , a * a + 2 * a * b + b * b) #this expression is simplified to 2ab
is_equal((a + b) * (a + b) , a * a + a * b + a * b + b * b) #this expression is expended Aka ab + ab

a not equal to b
a.b not equal to 2.a.b
a + b equal to b + a
2.a + 2.b equal to 2.b + 2.a
a.a + a.b equal to a.a + a.b
a.a + a.b + b.a + b.b not equal to a.a + 2.a.b + b.b
a.a + a.b + b.a + b.b equal to a.a + a.b + a.b + b.b


In [9]:
#a complex expression

expr_1 = grad(a * grad(a * b))

"""
Steps
(1) A = ∇(a⋅∇(a⋅b))
(2) A = ∇(a⋅(a⋅∇b + b⋅∇a))
(3) A = ∇(a⋅a⋅∇b + a.b⋅∇a))
(4) A = ∇(a⋅a⋅∇b) + ∇(a.b⋅∇a)
(5) A = a.∇(a⋅∇b) + a⋅∇b.∇(a) + a.∇(b⋅∇a) + b⋅∇a.∇a
(6) A = a.a.∇(∇b) + a.∇a.∇b + a⋅∇b.∇(a) + a.b.∇(∇a) + a.∇a.∇b + b⋅∇a.∇a
"""

#2
expr_2 = grad(a * (a * grad(b) +  b * grad(a)))
is_equal(expr_1, expr_2)

#3
expr_2 = grad(a * a * grad(b) + a * b * grad(a))
is_equal(expr_1, expr_2)

#4
expr_2 = grad(a * a * grad(b)) + grad(a * b * grad(a))
is_equal(expr_1, expr_2)

#5
expr_2 = a * grad(a * grad(b)) + a * grad(b) * grad(a) + a * grad(b * grad(a)) + b * grad(a) * grad(a)
is_equal(expr_1, expr_2)

#6
expr_2 = a * a * grad(grad(b)) + a * grad(b) * grad(a) + a * grad(b) * grad(a) + a * grad(b * grad(a)) + b * grad(a) * grad(a)
is_equal(expr_1, expr_2)



Grad(a.Grad(a.b)) equal to Grad(a.a.Grad(b) + a.b.Grad(a))
Grad(a.Grad(a.b)) equal to Grad(a.a.Grad(b) + a.b.Grad(a))
Grad(a.Grad(a.b)) equal to Grad(a.a.Grad(b)) + Grad(a.b.Grad(a))
Grad(a.Grad(a.b)) equal to a.Grad(a.Grad(b)) + a.Grad(b).Grad(a) + a.Grad(b.Grad(a)) + b.Grad(a).Grad(a)
Grad(a.Grad(a.b)) equal to a.a.Grad(Grad(b)) + a.Grad(b).Grad(a) + a.Grad(b).Grad(a) + a.Grad(b.Grad(a)) + b.Grad(a).Grad(a)
